In [1]:
import pyodbc
import pandas as pd

# Connection details
conn_str = (
    r'DRIVER={SQL Server};'
    r'SERVER=mlinfomap.net;'
    r'DATABASE=AI_ML_VTS;'
    r'UID=sa;'
    r'PWD=Intel@1968;'
)

# Establish connection
conn = pyodbc.connect(conn_str)

# Execute the stored procedure
query = """
EXEC GetTransporterViolationsUsingInvoice_Fast;
"""
df = pd.read_sql(query, conn)

# Close connection
conn.close()

# Calculate additional features
df['ViolationDensity'] = df['Total_Violations'] / df['Truck_Count']
df['ViolationRatePerInvoice'] = df['Total_Violations'] / df['Invoice_Count']

# Define target variable
df['Top10Transporter'] = df['Total_Violations'] >= df['Total_Violations'].nlargest(10).min()


C:\Users\MLINF\AppData\Local\Temp\ipykernel_672\1605225240.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Features and target
X = df[['Truck_Count', 'Invoice_Count', 'Route_Violation_Count', 
        'Speed_Violation_Count', 'Stoppage_Violation_Count', 
        'Night_Violation_Count']]
y = df['Total_Violations']  # Predict total violations

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 3936.620433333333


In [3]:
def forecast_for_future(month, year, historical_data, model):
    # Generate future data
    future_data = pd.DataFrame({
        'Truck_Count': [100] * 10,  # Placeholder values
        'Invoice_Count': [2000] * 10,
        'Route_Violation_Count': [50] * 10,
        'Speed_Violation_Count': [10] * 10,
        'Stoppage_Violation_Count': [30] * 10,
        'Night_Violation_Count': [20] * 10
    })
    
    # Make predictions
    future_data['Predicted_Violations'] = model.predict(future_data)
    
    # Add transporter names for demonstration
    future_data['TransporterName'] = [f'Transporter {i}' for i in range(1, 11)]
    
    # Rank by predicted violations
    top_10 = future_data.nlargest(10, 'Predicted_Violations')
    
    return top_10[['TransporterName', 'Predicted_Violations']]

# Example usage
future_month = 8  # August
future_year = 2024
top_10_transporters = forecast_for_future(future_month, future_year, df, model)

print(top_10_transporters)


  TransporterName  Predicted_Violations
0   Transporter 1                193.08
1   Transporter 2                193.08
2   Transporter 3                193.08
3   Transporter 4                193.08
4   Transporter 5                193.08
5   Transporter 6                193.08
6   Transporter 7                193.08
7   Transporter 8                193.08
8   Transporter 9                193.08
9  Transporter 10                193.08
